In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd


# for later
from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_excel('./data/TCM_Confidential.xlsx', sheet_name=2, header=3)

In [3]:
df = df.transpose()